In [1]:
import math
from itertools import combinations
import numpy as np
import jinja2
import pandas as pd
import scipy

In [2]:
DATAFILE = 'contest_data.csv'

In [3]:
class Contest:
    def __init__(self, filename, alpha=.1):
        self.filename = filename
        self.alpha = alpha
        self.load_df()

    def load_df(self):
        self.df = pd.read_csv(self.filename, index_col='Учасники')
        
    @property
    def headers(self):
        return self.df.columns # повертає список членів журі

    @property
    def entries(self):
        return self.df.itertuples()

    @property
    def m(self):
        return self.df.shape[0] # к-ть учасників конкурсу

    @property
    def n(self):
        return self.df.shape[1] # к-ть членів журі
    
    @property
    def k(self):
        return self.m - 2
    
    @property
    def concordance(self):
        S_prime_ = self.df.sum(axis=1)
        di_ = S_prime_ - S_prime_.mean()
        di_square_ = di_**2
        S = di_square_.sum()
        return 12 * S / (self.m*self.n**2*(self.m**2 - 1))

    @property
    def concordance_2(self):
        S_prime_ = self.df.sum(axis=1)
        di_ = S_prime_ - S_prime_.mean()
        di_square_ = di_**2
        sumD = di_square_.sum()
        return 120 * sumD / ((self.n**2*(self.m**3 - self.m) - self.n) *(self.m*3 - self.m))


    @property
    def distribution(self):
        df = pd.read_csv('contest_data.csv')
        alpha = 0.05
        spearman_corr, _ = scipy.stats.spearmanr(df)
        _, p_value = scipy.stats.ttest_1samp(spearman_corr, 0)
        if p_value.any() < alpha:
            return "є значущою"
        else:
            return "не є значущою"
    
    @property
    def spearman(self):
        t = -scipy.stats.t.ppf(self.alpha/2, self.k) # Why -2.35??
        rows = []
        for ex1, ex2 in combinations(self.df.columns, 2):
            rho_ij = 1 - 6 * ((self.df[ex1] - self.df[ex2])**2).sum() / (self.m**3 - self.m)
            rho_ij = round(rho_ij, 1)
            
            T_kp = t * math.sqrt((1 - rho_ij**2)/(self.m - 2))
            T_kp = round(T_kp, 9)
            
            importance = "+" if abs(rho_ij) > T_kp else "-"
            
            i = self.df.columns.get_loc(ex1) + 1
            j = self.df.columns.get_loc(ex2) + 1
            rows.append((i, j, rho_ij, round(t, 2), T_kp, importance))
            
        return pd.DataFrame(rows, columns=['i', 'j', 'rho_ij', 't', 'T_kp', 'importance'])

    @property
    def rating(self):
        return self.df.mean(axis=1).rank(ascending=False).astype(int)
        
    @property
    def dissidents(self):
        conc = self.concordance
        ex = "--"
        dis = []
        for i in range(len(self.df.columns)):
            dis.append({
                "expert": ex if i else "--", 
                "concordance": round(self.concordance, 4),
                "concordance_change" : round(self.concordance - dis[-1]["concordance"], 4) if i else 0,
                "importance" : self.distribution,
                "rating": self.rating,
                "winner_change" : ("Так" if (self.rating.idxmax() == dis[-1]["rating"].idxmax()) else "Y") if i else "--",
                "change_amount" : sum(self.rating == dis[0]["rating"]) if i else "--",
            })
            ex = self.df.columns[-1]
            self.df.pop(ex)
        self.load_df()
        return dis
    

In [4]:
environment = jinja2.Environment(loader=jinja2.FileSystemLoader(''))
#DATAFILE = 'test_2.csv'
template = environment.get_template('template.html')
contest = Contest(DATAFILE)
with open('report.html', 'w', encoding='utf-8') as report:
    report.write(template.render(contest=contest))


c:\Users\38096\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
c:\Users\38096\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
c:\Users\38096\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
c:\Users\38096\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ig

In [5]:
for row in contest.spearman.iterrows():
    for cell in row[1]:
        print(cell, end='\t')
    print()

1	2	0.1	2.35	1.351904368	-	
1	3	0.8	2.35	0.815229008	-	
1	4	0.4	2.35	1.245282879	-	
2	3	-0.4	2.35	1.245282879	-	
2	4	0.8	2.35	0.815229008	-	
3	4	-0.1	2.35	1.351904368	-	


In [6]:
contest.df

,Директор,Заступник_1,Заступник_2,Методист
Учасники,,,,
8-А,1,3,2,2
8-Б,2,4,1,3
8-В,3,2,3,2
9-А,4,1,5,1
9-Б,5,5,4,4
